In [1]:
#| default_exp core

In [2]:
from __future__ import annotations
import duckdb
from duckdb import DuckDBPyConnection
from duckdb import DuckDBPyRelation
from typing import List, Dict, Optional, Union, Any, Tuple, Set
from fastcore.all import store_attr, patch, L
import numpy as np
import pandas as pd
from dataclasses import field, make_dataclass

from functools import wraps, partial

In [3]:
db = duckdb.connect('../data/chinook.duckdb')
db

In [4]:
#| export
@patch(as_prop=True)
def table_names(self: DuckDBPyConnection): 
    return [r[0] for r in db.sql(f"SELECT table_name FROM duckdb_tables() WHERE schema_name = current_schema()").fetchall()]
@patch(as_prop=True)
def view_names(self: DuckDBPyConnection): 
    return [r[0] for r in db.sql(f"SELECT view_name FROM duckdb_views() WHERE schema_name = current_schema() and internal = False").fetchall()]
@patch(as_prop=True)
def function_names(self: DuckDBPyConnection): 
    return [r[0] for r in db.sql(f"SELECT function_name FROM duckdb_functions() WHERE schema_name = current_schema() and internal=False").fetchall()]

In [5]:
db.table_names, db.view_names, db.function_names

(['Album',
  'Artist',
  'Customer',
  'Employee',
  'Genre',
  'Invoice',
  'InvoiceLine',
  'MediaType',
  'Playlist',
  'PlaylistTrack',
  'Track'],
 ['a_view', 'temp_view'],
 [])

In [6]:
album = db.sql("select * from Album").set_alias('Album')
a_view = db.sql("select * from a_view")
a_view.show()

┌─────────┬──────────────────────┬─────────┬───┬──────────────────────┬──────────────┬──────────┬───────────────┐
│ TrackId │         Name         │ AlbumId │ … │       Composer       │ Milliseconds │  Bytes   │   UnitPrice   │
│  int32  │       varchar        │  int32  │   │       varchar        │    int32     │  int32   │ decimal(10,2) │
├─────────┼──────────────────────┼─────────┼───┼──────────────────────┼──────────────┼──────────┼───────────────┤
│       1 │ For Those About To…  │       1 │ … │ Angus Young, Malco…  │       343719 │ 11170334 │          0.98 │
│       2 │ Balls to the Wall    │       2 │ … │ NULL                 │       342562 │  5510424 │          0.98 │
│       3 │ Fast As a Shark      │       3 │ … │ F. Baltes, S. Kauf…  │       230619 │  3990994 │          0.98 │
│       4 │ Restless and Wild    │       3 │ … │ F. Baltes, R.A. Sm…  │       252051 │  4331779 │          0.98 │
│       5 │ Princess of the Dawn │       3 │ … │ Deaffy & R.A. Smit…  │       375418 │  

In [7]:
#| export

@patch(as_prop=True) # alias for alias
def name(self:DuckDBPyRelation): return self.alias

@patch # use __getitem__ as select
def __getitem__(self:DuckDBPyRelation, idxs) -> DuckDBPyRelation:
    return self.select(*idxs) if isinstance(idxs, Union[List, Set, Tuple]) else self.select(idxs)
@patch 
def to_recs(self:DuckDBPyRelation) -> List[Dict[str, Any]]:
    return self.df().to_dict(orient='records')

@patch 
def q(self:DuckDBPyConnection, *args, **kwargs) -> List[Dict[str, Any]]:
    return self.sql(*args, **kwargs).to_recs()

In [8]:
db.q(f"SELECT table_name FROM duckdb_tables() WHERE schema_name = current_schema()")

[{'table_name': 'Album'},
 {'table_name': 'Artist'},
 {'table_name': 'Customer'},
 {'table_name': 'Employee'},
 {'table_name': 'Genre'},
 {'table_name': 'Invoice'},
 {'table_name': 'InvoiceLine'},
 {'table_name': 'MediaType'},
 {'table_name': 'Playlist'},
 {'table_name': 'PlaylistTrack'},
 {'table_name': 'Track'}]

In [9]:
db.view_names

['a_view', 'temp_view']

In [10]:
#| export

@patch # use __getitem__ as select
def __getitem__(self:DuckDBPyRelation, idxs):
    return self.select(*idxs) if isinstance(idxs, Union[List, Set, Tuple]) else self.select(idxs)

In [11]:
album = db.sql("select * from Album")
album['AlbumId', 'Title']

┌─────────┬─────────────────────────────────────────────────────────────────────┐
│ AlbumId │                                Title                                │
│  int32  │                               varchar                               │
├─────────┼─────────────────────────────────────────────────────────────────────┤
│       1 │ For Those About To Rock We Salute You                               │
│       2 │ Balls to the Wall                                                   │
│       3 │ Restless and Wild                                                   │
│       4 │ Let There Be Rock                                                   │
│       5 │ Big Ones                                                            │
│       6 │ Jagged Little Pill                                                  │
│       7 │ Facelift                                                            │
│       8 │ Warner 25 Anos                                                      │
│       9 │ Play

In [12]:
album.columns, album.types, album.name

(['AlbumId', 'Title', 'ArtistId'],
 [INTEGER, VARCHAR, INTEGER],
 'unnamed_relation_94551e7d8c2ca735')

In [13]:
#column_name, column_type, notnull, dflt_value, pk
db.sql(f"PRAGMA table_info=album").fetchall() 

[(0, 'AlbumId', 'INTEGER', True, None, True),
 (1, 'Title', 'VARCHAR', True, None, False),
 (2, 'ArtistId', 'INTEGER', True, None, False)]

In [14]:
#| export

@patch
def datamodel(self: DuckDBPyConnection, table_name:str) ->List[Dict]:
    info =  self.sql(f"PRAGMA table_info='{table_name}'").fetchall()
    return [{'name': r[1], 'type': r[2], 'nullable': not r[3], 'default': r[4], 'pk': r[5]} for r in info]

In [15]:
#| export

from dataclasses import field, make_dataclass
def convertTypes(s:str)->type:
    d = {
        # Built-in types
        'BOOLEAN': bool,
        'BLOB': bytearray,  # For bytes, bytearray can be used in Python
        'DOUBLE': float,
        'BIGINT': int,
        'VARCHAR': str,
    
        # NumPy DTypes
        'FLOAT': np.float32,
        'DOUBLE': np.float64,
        'SMALLINT': np.int16,
        'INTEGER': np.int32,
        'BIGINT': np.int64,
        'TINYINT': np.int8,
        'USMALLINT': np.uint16,
        'UINTEGER': np.uint32,
        'UBIGINT': np.uint64,
        'UTINYINT': np.uint8
    }
    if s in d: return d[s]
    if s[:7]=='DECIMAL': return float
    return None


In [16]:
#| export

@patch
def _metadata(self: DuckDBPyConnection,name:str, type:str='table') -> Dict:
    table = (self.table(name) if type == 'table' else self.view(name))
    query = f"select comment from duckdb_{type}s() "
    where = f"where {type}_name = '{name}' and "
    where +=f"schema_name = current_schema() and internal = False"
    meta = self.sql(query+where).to_recs()[0]
    meta['row_count'] = table.shape[0]
    meta['col_count'] = table.shape[1]  
    return meta
@patch
def metadata(self: DuckDBPyConnection, name:str) -> Dict:
    if name in self.table_names: return self._metadata(name, 'table')
    else: return self._metadata(name, 'view')

In [17]:
#add comment to table information_schema in duckdb
db.sql("COMMENT ON TABLE Album IS 'Album table'")
db.metadata('Album')

{'comment': 'Album table', 'row_count': 347, 'col_count': 3}

In [18]:
db.sql("COMMENT ON VIEW a_view IS 'Just a view'")
db.metadata('a_view')

{'comment': 'Just a view', 'row_count': 3503, 'col_count': 9}

In [19]:
#| export

@patch
def dataclass(self: DuckDBPyConnection, table_name:str, pref='', suf='') -> type:
   fields = self.datamodel(table_name)
   fields = [(pref+f['name']+suf, convertTypes(f['type']) if not f['nullable'] else convertTypes(f['type'])|None , field(default=f['default'])) for f in fields]
   return make_dataclass(table_name, fields)

In [20]:
db.dataclass('a_view')

__main__.a_view

In [21]:
from fastcore.xtras import hl_md, dataclass_src

In [22]:
artist_dc = db.dataclass('Artist')
src = dataclass_src(artist_dc)
hl_md(src, 'python')

```python
@dataclass
class Artist:
    ArtistId: int32 = None
    Name: str | None = None

```

In [23]:
acdc = db.q(f"select * from artist where artist.Name like 'AC/%'")
acdc_object = artist_dc(**acdc[0])
acdc_object

Artist(ArtistId=1, Name='AC/DC')

I you know the name of the tables and columns, duckdb Python API has a nice scan feature

In [24]:
a = db.table('Album') # you must know the name of the tables
a.ArtistId # you must know the name of the column

┌────────────┐
│  ArtistId  │
│   int32    │
├────────────┤
│          1 │
│          2 │
│          2 │
│          1 │
│          3 │
│          4 │
│          5 │
│          6 │
│          7 │
│          8 │
│          · │
│          · │
│          · │
│        267 │
│        268 │
│        269 │
│        270 │
│        271 │
│        226 │
│        272 │
│        273 │
│        274 │
│        275 │
├────────────┤
│  347 rows  │
│ (20 shown) │
└────────────┘

In [25]:
db.sql("select * from a where a.ArtistId = 1")

┌─────────┬───────────────────────────────────────┬──────────┐
│ AlbumId │                 Title                 │ ArtistId │
│  int32  │                varchar                │  int32   │
├─────────┼───────────────────────────────────────┼──────────┤
│       1 │ For Those About To Rock We Salute You │        1 │
│       4 │ Let There Be Rock                     │        1 │
└─────────┴───────────────────────────────────────┴──────────┘

In [26]:
#| export

def noop(*args, **kwargs): return None
def identity(x): return x

In [27]:
#| export

class _Getter: 
    """ A Getter utility check https://github.com/AnswerDotAI/fastlite """
    def __init__(self, name:str='', type:str='', dir:List=[], get=noop): store_attr()    
    def __dir__(self): return self.dir
    def __str__(self): return ", ".join(dir(self))
    def __repr__(self): return f"{self.type}::{self.name}: {str(self)}"
    def __contains__(self, s:str): return s in dir(self)
    def __getitem__(self, k): return self.get(k)
    def __getattr__(self, k):
        if k[0]!='_': return self.get(k)
        else: raise AttributeError 

In [28]:
#| export
def custom_dir(c, add): return dir(type(c)) + list(c.__dict__.keys()) + add

class Database:
    def _current(self: DuckDBPyConnection): return self.sql('select current_catalog, current_schema').fetchone()

    def __init__(self, *args, **kwargs):
        self.conn = duckdb.connect(*args, **kwargs) # original connection object is often called
        self.catalog, self.schema = self.conn.sql('select current_catalog, current_schema').fetchone()
        self.tables =  {}
        self.views = {}
        self.xtra = ['query', 'cursor', 'execute', 'fetchone', 'fetchall', 'sql', 'close', 'table_names', 'view_names', 'dataclass', 'metadata']

    def __dir__(self:Database): return custom_dir(self, self.xtra)  
    def __repr__(self): return f'{self.__class__.__name__} ({self.catalog}::{self.schema})'

    def __getattr__(self:Database, k):
        if k in self.xtra: return getattr(self.conn, k)
        if not isinstance(k, str) and k[0]=='_': raise AttributeError
        return self.table(k, include_views=True)        
          
    def __getitem__(self, k): return self.table(k)
    
    
    def table(self, k):
        if isinstance(k, Union[List, Set, Tuple]): return [self.table(ki) for ki in k]
        if not isinstance(k,str): raise AttributeError
        if self.conn.table(k): 
            tbl = Table(self.conn.table(k), self, k)
            self.tables= {**self.tables, **{k: tbl}}
            return tbl
    
    def view(self, k):
        if isinstance(k, Union[List, Set, Tuple]): return [self.view(ki) for ki in k]
        if not isinstance(k,str): raise AttributeError
        if self.conn.view(k): 
            vw = View(rel=self.conn.view(k), db=self, name=k)
            self.views= {**self.views, **{k: vw}}
            return vw
    
        
    @property # tables
    def t(self:Database): return _Getter('Tables', 'Table', self.conn.table_names, self.table)
    @property # views
    def v(self:Database): return _Getter('Views', 'View', self.conn.view_names, self.view)
    @property # functions
    def fns(self:Database): raise NotImplementedError
    # def fns(self:Database): return _Getter(self, f"SELECT function_name FROM duckdb_functions() WHERE schema_name = '{self.schema}' and internal = False")
    @property # secrets
    def shh(self:Database): raise NotImplementedError
    # def shh(self:Database): return _Getter(self, f"SELECT name FROM duckdb_secrets()")

    def q(self:Database, query:str): return self.conn.sql(query).to_recs()
    

In [29]:
#| export
class Table:
    def __init__(self, rel:duckdb.DuckDBPyRelation, db:Database, name:str):
        store_attr()
        self.type='Table'
        self.cls = db.dataclass(name)
        self.meta = db.metadata(name)
        self.rel = self.rel.set_alias(name)    
        self.c = _Getter('Columns', 'Column', rel.columns, rel.select)
    def __dir__(self): return custom_dir(self, [s for s in dir(self.rel) if s[0]!= '_'])
    def __getattr__(self, k): 
        if k[0]=='_': raise AttributeError
        return getattr(self.rel, k)
    def __getitem__(self, idx): return self.rel[idx]
    def __str__(self): return f'"{self.db.schema}"."{self.name}"'
    def __repr__(self): return f'<{self.__class__.__name__}  {self.name} {dir(self.c)}>'
    def _repr_markdown_(self):
        markdown = f"**{self.__class__.__name__}: {self.name}** ({self.meta['row_count']} rows, {self.meta['col_count']} cols)\n"
        if self.meta['comment']: markdown += f"> {self.meta['comment']}"
        markdown = (self.rel.df()).head(10).to_markdown(index=False)
        return markdown
        
class View(Table):
    def __init__(self, rel:duckdb.DuckDBPyRelation, db:Database, name:str):
        super().__init__(rel, db, name) 
        self.type = 'View'
    def __getattr__(self, k): # must be overriden otherwise doesn't work
        if k[0]=='_': raise AttributeError
        return getattr(self.rel, k)

In [30]:
db.close()

In [31]:
db = Database('../data/chinook.duckdb')
db

Database (chinook::main)

In [32]:
db.view_names

['a_view', 'temp_view']

In [33]:
db.view('a_view')

|   TrackId | Name                                    |   AlbumId |   MediaTypeId |   GenreId | Composer                                                               |   Milliseconds |    Bytes |   UnitPrice |
|----------:|:----------------------------------------|----------:|--------------:|----------:|:-----------------------------------------------------------------------|---------------:|---------:|------------:|
|         1 | For Those About To Rock (We Salute You) |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         343719 | 11170334 |        0.98 |
|         2 | Balls to the Wall                       |         2 |             2 |         1 |                                                                        |         342562 |  5510424 |        0.98 |
|         3 | Fast As a Shark                         |         3 |             2 |         1 | F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman                    |         230619 |  3990994 |        0.98 |
|         4 | Restless and Wild                       |         3 |             2 |         1 | F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman |         252051 |  4331779 |        0.98 |
|         5 | Princess of the Dawn                    |         3 |             2 |         1 | Deaffy & R.A. Smith-Diesel                                             |         375418 |  6290521 |        0.98 |
|         6 | Put The Finger On You                   |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         205662 |  6713451 |        0.98 |
|         7 | Let's Get It Up                         |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         233926 |  7636561 |        0.98 |
|         8 | Inject The Venom                        |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         210834 |  6852860 |        0.98 |
|         9 | Snowballed                              |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         203102 |  6599424 |        0.98 |
|        10 | Evil Walks                              |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         263497 |  8611245 |        0.98 |

In [34]:
db.t.Artist

|   ArtistId | Name                 |
|-----------:|:---------------------|
|          1 | AC/DC                |
|          2 | Accept               |
|          3 | Aerosmith            |
|          4 | Alanis Morissette    |
|          5 | Alice In Chains      |
|          6 | Antônio Carlos Jobim |
|          7 | Apocalyptica         |
|          8 | Audioslave           |
|          9 | BackBeat             |
|         10 | Billy Cobham         |

In [35]:
a = db.t.Album
a

|   AlbumId | Title                                 |   ArtistId |
|----------:|:--------------------------------------|-----------:|
|         1 | For Those About To Rock We Salute You |          1 |
|         2 | Balls to the Wall                     |          2 |
|         3 | Restless and Wild                     |          2 |
|         4 | Let There Be Rock                     |          1 |
|         5 | Big Ones                              |          3 |
|         6 | Jagged Little Pill                    |          4 |
|         7 | Facelift                              |          5 |
|         8 | Warner 25 Anos                        |          6 |
|         9 | Plays Metallica By Four Cellos        |          7 |
|        10 | Audioslave                            |          8 |

In [36]:
a = db.table('a_view')
type(a)

__main__.Table

In [37]:
track = db.table('Track')
track

|   TrackId | Name                                    |   AlbumId |   MediaTypeId |   GenreId | Composer                                                               |   Milliseconds |    Bytes |   UnitPrice |
|----------:|:----------------------------------------|----------:|--------------:|----------:|:-----------------------------------------------------------------------|---------------:|---------:|------------:|
|         1 | For Those About To Rock (We Salute You) |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         343719 | 11170334 |        0.98 |
|         2 | Balls to the Wall                       |         2 |             2 |         1 |                                                                        |         342562 |  5510424 |        0.98 |
|         3 | Fast As a Shark                         |         3 |             2 |         1 | F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman                    |         230619 |  3990994 |        0.98 |
|         4 | Restless and Wild                       |         3 |             2 |         1 | F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman |         252051 |  4331779 |        0.98 |
|         5 | Princess of the Dawn                    |         3 |             2 |         1 | Deaffy & R.A. Smith-Diesel                                             |         375418 |  6290521 |        0.98 |
|         6 | Put The Finger On You                   |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         205662 |  6713451 |        0.98 |
|         7 | Let's Get It Up                         |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         233926 |  7636561 |        0.98 |
|         8 | Inject The Venom                        |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         210834 |  6852860 |        0.98 |
|         9 | Snowballed                              |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         203102 |  6599424 |        0.98 |
|        10 | Evil Walks                              |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         263497 |  8611245 |        0.98 |

In [38]:
db.view('a_view')

|   TrackId | Name                                    |   AlbumId |   MediaTypeId |   GenreId | Composer                                                               |   Milliseconds |    Bytes |   UnitPrice |
|----------:|:----------------------------------------|----------:|--------------:|----------:|:-----------------------------------------------------------------------|---------------:|---------:|------------:|
|         1 | For Those About To Rock (We Salute You) |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         343719 | 11170334 |        0.98 |
|         2 | Balls to the Wall                       |         2 |             2 |         1 |                                                                        |         342562 |  5510424 |        0.98 |
|         3 | Fast As a Shark                         |         3 |             2 |         1 | F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman                    |         230619 |  3990994 |        0.98 |
|         4 | Restless and Wild                       |         3 |             2 |         1 | F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman |         252051 |  4331779 |        0.98 |
|         5 | Princess of the Dawn                    |         3 |             2 |         1 | Deaffy & R.A. Smith-Diesel                                             |         375418 |  6290521 |        0.98 |
|         6 | Put The Finger On You                   |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         205662 |  6713451 |        0.98 |
|         7 | Let's Get It Up                         |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         233926 |  7636561 |        0.98 |
|         8 | Inject The Venom                        |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         210834 |  6852860 |        0.98 |
|         9 | Snowballed                              |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         203102 |  6599424 |        0.98 |
|        10 | Evil Walks                              |         1 |             1 |         1 | Angus Young, Malcolm Young, Brian Johnson                              |         263497 |  8611245 |        0.98 |

In [39]:
db.t.Playlist

|   PlaylistId | Name         |
|-------------:|:-------------|
|            1 | Music        |
|            2 | Movies       |
|            3 | TV Shows     |
|            4 | Audiobooks   |
|            5 | 90’s Music   |
|            6 | Audiobooks   |
|            7 | Movies       |
|            8 | Music        |
|            9 | Music Videos |
|           10 | TV Shows     |

In [40]:
artist = db.t.Artist
db.sql(f"select * from artist")

┌──────────┬────────────────────────────────────────────────────────────────────────────────────┐
│ ArtistId │                                        Name                                        │
│  int32   │                                      varchar                                       │
├──────────┼────────────────────────────────────────────────────────────────────────────────────┤
│        1 │ AC/DC                                                                              │
│        2 │ Accept                                                                             │
│        3 │ Aerosmith                                                                          │
│        4 │ Alanis Morissette                                                                  │
│        5 │ Alice In Chains                                                                    │
│        6 │ Antônio Carlos Jobim                                                               │
│        7 │ Apocaly

In [41]:
db.sql(f"select * from {artist}")

┌──────────┬────────────────────────────────────────────────────────────────────────────────────┐
│ ArtistId │                                        Name                                        │
│  int32   │                                      varchar                                       │
├──────────┼────────────────────────────────────────────────────────────────────────────────────┤
│        1 │ AC/DC                                                                              │
│        2 │ Accept                                                                             │
│        3 │ Aerosmith                                                                          │
│        4 │ Alanis Morissette                                                                  │
│        5 │ Alice In Chains                                                                    │
│        6 │ Antônio Carlos Jobim                                                               │
│        7 │ Apocaly

In [42]:
a = db.t.Artist
a

|   ArtistId | Name                 |
|-----------:|:---------------------|
|          1 | AC/DC                |
|          2 | Accept               |
|          3 | Aerosmith            |
|          4 | Alanis Morissette    |
|          5 | Alice In Chains      |
|          6 | Antônio Carlos Jobim |
|          7 | Apocalyptica         |
|          8 | Audioslave           |
|          9 | BackBeat             |
|         10 | Billy Cobham         |

In [43]:
# db.sql(f"select * from a")

#### Export

In [44]:
#| hide
import nbdev; nbdev.nbdev_export()